In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')

In [3]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [4]:
app_train_origin =  pd.read_csv('./home-credit-default-risk/application_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/application_test.csv')
app_train_origin.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
display(app_train['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_train['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_train['DAYS_EMPLOYED']>0).sum()))

 365243    55374
-200         156
-224         152
-230         151
-199         151
           ...  
-13961         1
-11827         1
-10176         1
-9459          1
-8694          1
Name: DAYS_EMPLOYED, Length: 12574, dtype: int64

正の値の割合: 0.1801
正の値の個数: 55374


### one-hot-encoding

In [7]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
### CODE_GENDER
app_train['CODE_GENDER'].replace(['XNA'], ['M'], inplace=True)
app_test['CODE_GENDER'].replace(['XNA'], ['M'], inplace=True)
app_train['CODE_GENDER'].replace(['F', 'M'], [0, 1], inplace=True)
app_test['CODE_GENDER'].replace(['F', 'M'], [0, 1], inplace=True)
#encoding_columns.remove('CODE_GENDER')
### NAME_INCOME_TYPE
app_train['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
app_test['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
app_train['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
#encoding_columns.remove('NAME_INCOME_TYPE')
### NAME_FAMILY_STATUS
app_train['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
app_test['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
app_train['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)
app_test['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)
#encoding_columns.remove('NAME_FAMILY_STATUS')

In [9]:
# one-hotencoding
encoding_columns = []

for col in app_test.columns:
    if app_test[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns.append('CODE_GENDER')
encoding_columns.append('NAME_INCOME_TYPE')
encoding_columns.append('NAME_FAMILY_STATUS')

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_EDUCATION_TYPE
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [10]:
for col in encoding_columns:
    app_train = pd.get_dummies(app_train, columns=[col], dummy_na=True)
    app_test = pd.get_dummies(app_test, columns=[col], dummy_na=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [11]:
### 
for col in app_train.columns:
    app_train[col] = app_train[col].replace('\s', '_', regex=True)
    app_train[col] = app_train[col].replace(':', '_', regex=True)
    app_train[col] = app_train[col].replace('/', '_', regex=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [12]:
### 
for col in app_test.columns:
    app_test[col] = app_test[col].replace('\s', '_', regex=True)
    app_test[col] = app_test[col].replace(':', '_', regex=True)
    app_test[col] = app_test[col].replace('/', '_', regex=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,1,0,0,0,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,1,0,0,0,0,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,1,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,1,0,0,0,0


In [13]:
len(app_train.columns)

259

In [14]:
len(app_test.columns)

258

In [15]:
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_INCOME_TYPE_4.0,NAME_INCOME_TYPE_5.0,NAME_INCOME_TYPE_6.0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0.0,NAME_FAMILY_STATUS_1.0,NAME_FAMILY_STATUS_2.0,NAME_FAMILY_STATUS_3.0,NAME_FAMILY_STATUS_4.0,NAME_FAMILY_STATUS_nan
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0,1,0,0,0,0,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0,0,0,0,1,0,0,0,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0,0,0,0,1,0,0,0,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0,1,0,0,0,0,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,0,0,0,0,1,0,0,0,0,0


In [16]:
app_train = app_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
app_test = app_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### app特徴量作成

In [17]:
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

In [18]:
docs = [_f for _f in app_train.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_train.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_train['ORGANIZATION_TYPE'] = app_train_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_train[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_train['INCOME_div_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_train['INCOME_div_EMPLOYED'] = app_train['AMT_INCOME_TOTAL'] / app_train['DAYS_EMPLOYED']

#
app_train['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_ANNUITY']

app_train['NEW_CREDIT_TO_GOODS_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']

app_train['NEW_DOC_IND_KURT'] = app_train[docs].kurtosis(axis=1)

app_train['NEW_LIVE_IND_SUM'] = app_train[live].sum(axis=1)

app_train['NEW_INC_PER_CHLD'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['CNT_CHILDREN'])

app_train['NEW_INC_BY_ORG'] = app_train['ORGANIZATION_TYPE'].map(inc_by_org)

app_train['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

app_train['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_train['DAYS_EMPLOYED'] / (app_train['DAYS_BIRTH'] + 18*365)

app_train['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_train['DAYS_BIRTH'] / (1 + app_train['DAYS_EMPLOYED'])

app_train['NEW_INCOME_TO_ANNUITY_RATIO'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['AMT_ANNUITY'])

app_train['NEW_ANNUITY_TO_INCOME_RATIO'] = app_train['AMT_ANNUITY'] / (1 + app_train['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_train['NEW_EXT_SOURCES_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_train['EXT_SOURCE_mean'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_train['EXT_SOURCE_max'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_train['EXT_SOURCE_min'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_std'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_train['EXT_SOURCE_count'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_train['NEW_EXT_SOURCES_PROD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_train['NEW_EXT_SOURCES_MAD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_train['NEW_EXT_SOURCES_GEO'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_train['DAYS_EMPLOYED_div_BERTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_train['ANNUITY_div_INCOME'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_train['ANNUITY_div_CREDIT'] = app_train['AMT_ANNUITY'] / app_train['AMT_CREDIT']

### 
app_train['credit_goods_price_ratio'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']
 
app_train['credit_downpayment'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']

app_train['USER_AGE'] = abs(app_train['DAYS_BIRTH'])/364.25

app_train['NEW_CAR_TO_BIRTH_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_BIRTH']

app_train['NEW_CAR_TO_EMPLOY_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_PHONE_TO_BIRTH_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_BIRTH']

app_train['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_CREDIT_TO_INCOME_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_INCOME_TOTAL']

app_train['NEW_INCOME_CREDIT_PERC'] = app_train['AMT_INCOME_TOTAL'] / app_train['AMT_CREDIT']

app_train['NEW_INCOME_PER_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

### 2022/06/25
app_train['app_EXT_SOURCE_prod'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_2'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_2']
app_train['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = app_train['EXT_SOURCE_1'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = app_train['EXT_SOURCE_2'] * app_train['EXT_SOURCE_3']
app_train['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_1'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_2'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = app_train['EXT_SOURCE_3'] * app_train['DAYS_EMPLOYED']
app_train['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_1'] / app_train['DAYS_BIRTH']
app_train['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_2'] / app_train['DAYS_BIRTH']
app_train['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = app_train['EXT_SOURCE_3'] / app_train['DAYS_BIRTH']

### 2022/06/26
#app_train['app_most_popular_AMT_GOODS_PRICE'] = app_train['AMT_GOODS_PRICE'] \
#    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
#app_train['app_popular_AMT_GOODS_PRICE'] = app_train['AMT_GOODS_PRICE'] \
#    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
#app_train['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_BIRTH']
#app_train['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_EMPLOYED']
#app_train['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_BIRTH']
#app_train['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_EMPLOYED']
#app_train['app_DAYS_EMPLOYED_PERC'] = app_train['DAYS_EMPLOYED'] - app_train['DAYS_BIRTH']
#app_train['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']
#app_train['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = app_train['CNT_CHILDREN'] / app_train['CNT_FAM_MEMBERS']

### 
app_train.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_train.columns)

304

In [19]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [20]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 302) (61503, 302)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.758829	valid_1's auc: 0.744355
[200]	training's auc: 0.770282	valid_1's auc: 0.752125
[300]	training's auc: 0.780319	valid_1's auc: 0.758258
[400]	training's auc: 0.789328	valid_1's auc: 0.762774
[500]	training's auc: 0.7967	valid_1's auc: 0.765617
[600]	training's auc: 0.803214	valid_1's auc: 0.767493
[700]	training's auc: 0.809013	valid_1's auc: 0.768655
[800]	training's auc: 0.814534	valid_1's auc: 0.769543
[900]	training's auc: 0.819668	valid_1's auc: 0.77018
[1000]	training's auc: 0.82451	valid_1's auc: 0.770786
[1100]	training's auc: 0.829059	valid_1's auc: 0.771145
[1200]	training's auc: 0.833475	valid_1's auc: 0.771381
[1300]	training's auc: 0.837598	valid_1's auc: 0.771384
[1400]	training's auc: 0.841598	valid_1's auc: 0.771472
[1500]	training's auc: 0.845621	valid_1's au

In [21]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
48,EXT_SOURCE_mean,308790.949113,3119.898229
160,NEW_EXT_SOURCES_GEO,130238.175772,768.721642
162,NEW_EXT_SOURCES_MEDIAN,64955.152573,1746.048339
154,NEW_CREDIT_TO_ANNUITY_RATIO,64494.314278,4846.845580
10,ANNUITY_div_CREDIT,48383.429829,6688.071842
49,EXT_SOURCE_min,47423.612644,1565.068819
47,EXT_SOURCE_max,38713.765939,2325.247474
296,app_EXT_SOURCE_2_times_EXT_SOURCE_3,28399.558829,2095.607610
155,NEW_CREDIT_TO_GOODS_RATIO,28076.889760,1273.774998
45,EXT_SOURCE_3,25028.728452,702.911022


In [22]:
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

In [23]:
docs = [_f for _f in app_test.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_test.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_test['ORGANIZATION_TYPE'] = app_test_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_test[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_test['INCOME_div_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_test['INCOME_div_EMPLOYED'] = app_test['AMT_INCOME_TOTAL'] / app_test['DAYS_EMPLOYED']

#
app_test['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_ANNUITY']

app_test['NEW_CREDIT_TO_GOODS_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']

app_test['NEW_DOC_IND_KURT'] = app_test[docs].kurtosis(axis=1)

app_test['NEW_LIVE_IND_SUM'] = app_test[live].sum(axis=1)

app_test['NEW_INC_PER_CHLD'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['CNT_CHILDREN'])

app_test['NEW_INC_BY_ORG'] = app_test['ORGANIZATION_TYPE'].map(inc_by_org)

app_test['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

app_test['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_test['DAYS_EMPLOYED'] / (app_test['DAYS_BIRTH'] + 18*365)

app_test['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_test['DAYS_BIRTH'] / (1 + app_test['DAYS_EMPLOYED'])

app_test['NEW_INCOME_TO_ANNUITY_RATIO'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['AMT_ANNUITY'])

app_test['NEW_ANNUITY_TO_INCOME_RATIO'] = app_test['AMT_ANNUITY'] / (1 + app_test['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_test['NEW_EXT_SOURCES_MEDIAN'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_test['EXT_SOURCE_mean'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_test['EXT_SOURCE_max'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_test['EXT_SOURCE_min'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_test['EXT_SOURCE_std'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_test['EXT_SOURCE_count'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_test['NEW_EXT_SOURCES_PROD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_test['NEW_EXT_SOURCES_MAD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_test['NEW_EXT_SOURCES_GEO'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_test['DAYS_EMPLOYED_div_BERTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_test['ANNUITY_div_INCOME'] = app_test['AMT_ANNUITY'] / app_test['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_test['ANNUITY_div_CREDIT'] = app_test['AMT_ANNUITY'] / app_test['AMT_CREDIT']

###
app_test['credit_goods_price_ratio'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']
 
app_test['credit_downpayment'] = app_test['AMT_GOODS_PRICE'] - app_test['AMT_CREDIT']

app_test['USER_AGE'] = abs(app_test['DAYS_BIRTH'])/364.25

app_test['NEW_CAR_TO_BIRTH_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_BIRTH']

app_test['NEW_CAR_TO_EMPLOY_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_PHONE_TO_BIRTH_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_BIRTH']

app_test['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_CREDIT_TO_INCOME_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_INCOME_TOTAL']

app_test['NEW_INCOME_CREDIT_PERC'] = app_test['AMT_INCOME_TOTAL'] / app_test['AMT_CREDIT']

app_test['NEW_INCOME_PER_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

### 2022/06/25
app_test['app_EXT_SOURCE_prod'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_2'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_2']
app_test['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = app_test['EXT_SOURCE_1'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = app_test['EXT_SOURCE_2'] * app_test['EXT_SOURCE_3']
app_test['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_1'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_2'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = app_test['EXT_SOURCE_3'] * app_test['DAYS_EMPLOYED']
app_test['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_1'] / app_test['DAYS_BIRTH']
app_test['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_2'] / app_test['DAYS_BIRTH']
app_test['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = app_test['EXT_SOURCE_3'] / app_test['DAYS_BIRTH']

### 2022/06/26
#app_test['app_most_popular_AMT_GOODS_PRICE'] = app_test['AMT_GOODS_PRICE'] \
#    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
#app_test['app_popular_AMT_GOODS_PRICE'] = app_test['AMT_GOODS_PRICE'] \
#    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
#app_test['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_BIRTH']
#app_test['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_EMPLOYED']
#app_test['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_BIRTH']
#app_test['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_EMPLOYED']
#app_test['app_DAYS_EMPLOYED_PERC'] = app_test['DAYS_EMPLOYED'] - app_test['DAYS_BIRTH']
#app_test['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']
#app_test['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = app_test['CNT_CHILDREN'] / app_test['CNT_FAM_MEMBERS']


### 2022/06/27


### 
app_test.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_test.columns)

303

In [24]:
# データセット
x_test = app_test.drop(columns=['SK_ID_CURR'])
id_test = app_test[['SK_ID_CURR']]

for col in x_test.columns:
    if x_test[col].dtype=='O':
        x_test[col] = x_test[col].astype('category')

In [25]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [26]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032711
1,100005,0.069395
2,100013,0.014633
3,100028,0.044881
4,100038,0.167964


### CSVの読み込みと前処理

In [27]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,app_EXT_SOURCE_prod,app_EXT_SOURCE_1_times_EXT_SOURCE_2,app_EXT_SOURCE_1_times_EXT_SOURCE_3,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_times_DAYS_EMPLOYED,app_EXT_SOURCE_2_times_DAYS_EMPLOYED,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_PER_DAYS_BIRTH,app_EXT_SOURCE_3_PER_DAYS_BIRTH
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.003043,0.021834,0.011573,0.036649,-52.894548,-167.498254,-88.782372,-0.000009,-0.000028,-0.000015
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,NaN,0.193685,NaN,NaN,-369.785566,-739.227981,NaN,-0.000019,-0.000037,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,NaN,NaN,NaN,0.405575,NaN,-125.080219,-164.152505,NaN,-0.000029,-0.000038
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,NaN,NaN,NaN,NaN,NaN,-1976.692297,NaN,NaN,-0.000034,NaN
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,NaN,NaN,NaN,NaN,NaN,-980.478916,NaN,NaN,-0.000016,NaN


In [28]:
# 欠損値補完
app_train = app_train.fillna(-99999999)
app_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,app_EXT_SOURCE_prod,app_EXT_SOURCE_1_times_EXT_SOURCE_2,app_EXT_SOURCE_1_times_EXT_SOURCE_3,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_times_DAYS_EMPLOYED,app_EXT_SOURCE_2_times_DAYS_EMPLOYED,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_PER_DAYS_BIRTH,app_EXT_SOURCE_3_PER_DAYS_BIRTH
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,3.043194e-03,2.183445e-02,1.157334e-02,3.664867e-02,-5.289455e+01,-167.498254,-8.878237e+01,-8.776764e-06,-0.000028,-1.473161e-05
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,-1.000000e+08,1.936848e-01,-1.000000e+08,-1.000000e+08,-3.697856e+02,-739.227981,-1.000000e+08,-1.856650e-05,-0.000037,-1.000000e+08
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,-1.000000e+08,-1.000000e+08,-1.000000e+08,4.055749e-01,-1.000000e+08,-125.080219,-1.641525e+02,-1.000000e+08,-0.000029,-3.830551e-05
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,-1.000000e+08,-1.000000e+08,-1.000000e+08,-1.000000e+08,-1.000000e+08,-1976.692297,-1.000000e+08,-1.000000e+08,-0.000034,-1.000000e+08
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,-1.000000e+08,-1.000000e+08,-1.000000e+08,-1.000000e+08,-1.000000e+08,-980.478916,-1.000000e+08,-1.000000e+08,-0.000016,-1.000000e+08


In [29]:
# infをnanへ置換
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,app_EXT_SOURCE_prod,app_EXT_SOURCE_1_times_EXT_SOURCE_2,app_EXT_SOURCE_1_times_EXT_SOURCE_3,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_times_DAYS_EMPLOYED,app_EXT_SOURCE_2_times_DAYS_EMPLOYED,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_PER_DAYS_BIRTH,app_EXT_SOURCE_3_PER_DAYS_BIRTH
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.094803,0.594305,0.120057,0.125965,-1752.839149,-1839.104984,-371.521010,-0.000039,-0.000041,-0.000008
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.071345,0.164783,0.244619,0.126276,-2524.941212,-1303.408573,-1934.905690,-0.000031,-0.000016,-0.000024
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,NaN,NaN,NaN,0.427564,NaN,-3119.649689,-2723.799341,NaN,-0.000035,-0.000030
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.164177,0.267955,0.322119,0.312281,-981.019602,-951.057432,-1143.306119,-0.000038,-0.000036,-0.000044
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,NaN,0.086051,NaN,NaN,-442.899678,-932.680861,NaN,-0.000016,-0.000033,NaN


In [30]:
# 欠損値補完
app_test = app_test.fillna(-99999999)
app_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,app_EXT_SOURCE_prod,app_EXT_SOURCE_1_times_EXT_SOURCE_2,app_EXT_SOURCE_1_times_EXT_SOURCE_3,app_EXT_SOURCE_2_times_EXT_SOURCE_3,app_EXT_SOURCE_1_times_DAYS_EMPLOYED,app_EXT_SOURCE_2_times_DAYS_EMPLOYED,app_EXT_SOURCE_3_times_DAYS_EMPLOYED,app_EXT_SOURCE_1_PER_DAYS_BIRTH,app_EXT_SOURCE_2_PER_DAYS_BIRTH,app_EXT_SOURCE_3_PER_DAYS_BIRTH
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,9.480331e-02,5.943053e-01,1.200567e-01,1.259653e-01,-1.752839e+03,-1839.104984,-3.715210e+02,-3.911514e-05,-0.000041,-8.290606e-06
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,7.134451e-02,1.647825e-01,2.446191e-01,1.262757e-01,-2.524941e+03,-1303.408573,-1.934906e+03,-3.127714e-05,-0.000016,-2.396821e-05
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,-1.000000e+08,-1.000000e+08,-1.000000e+08,4.275637e-01,-1.000000e+08,-3119.649689,-2.723799e+03,-1.000000e+08,-0.000035,-3.049163e-05
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,1.641769e-01,2.679546e-01,3.221194e-01,3.122813e-01,-9.810196e+02,-951.057432,-1.143306e+03,-3.761691e-05,-0.000036,-4.383974e-05
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,-1.000000e+08,8.605055e-02,-1.000000e+08,-1.000000e+08,-4.428997e+02,-932.680861,-1.000000e+08,-1.550192e-05,-0.000033,-1.000000e+08


In [31]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [32]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [33]:
# infをnanへ置換
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau_balance = bureau_balance.replace([np.inf, -np.inf], np.nan)
pos_cash_balance = pos_cash_balance.replace([np.inf, -np.inf], np.nan)
credit_card_balance = credit_card_balance.replace([np.inf, -np.inf], np.nan)
previous_application = previous_application.replace([np.inf, -np.inf], np.nan)
installments_payments = installments_payments.replace([np.inf, -np.inf], np.nan)

pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


### pos_cashの紐づけ

In [34]:
baseline = Baseline('SK_ID_CURR')

pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [35]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [36]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [37]:
### infをnanへ置換
pos_ohe_agg = pos_ohe_agg.replace([np.inf, -np.inf], np.nan)
pos_ohe_agg.head()

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [38]:
df_train = pd.merge(app_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 336)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [39]:
### テーブル結合
df_test = pd.merge(app_test, pos_ohe_agg, on='SK_ID_CURR', how="left")
print(df_test.shape)
df_test.head()

(48744, 335)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [40]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [41]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [42]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [43]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [44]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [45]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 781.26 MB
Memory usage of optimization is 412372379.00 MB
Decreased by -52782956.8%
Memory usage of dataframe is 123.47 MB
Memory usage of optimization is 65317088.00 MB
Decreased by -52902505.6%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [46]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [47]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 334) (61503, 334)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.762301	valid_1's auc: 0.746659
[200]	training's auc: 0.775309	valid_1's auc: 0.755415
[300]	training's auc: 0.786134	valid_1's auc: 0.761801
[400]	training's auc: 0.795243	valid_1's auc: 0.766533
[500]	training's auc: 0.802812	valid_1's auc: 0.769561
[600]	training's auc: 0.809347	valid_1's auc: 0.771392
[700]	training's auc: 0.815375	valid_1's auc: 0.772893
[800]	training's auc: 0.820904	valid_1's auc: 0.773987
[900]	training's auc: 0.826136	valid_1's auc: 0.774947
[1000]	training's auc: 0.830976	valid_1's auc: 0.775543
[1100]	training's auc: 0.835452	valid_1's auc: 0.775876
[1200]	training's auc: 0.839779	valid_1's auc: 0.776125
[1300]	training's auc: 0.843856	valid_1's auc: 0.776451
[1400]	training's auc: 0.847777	valid_1's auc: 0.776563
[1500]	training's auc: 0.851608	valid_1'

In [48]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
56,EXT_SOURCE_mean,340425.267342,8062.892655
182,NEW_EXT_SOURCES_GEO,155927.165308,5855.517941
184,NEW_EXT_SOURCES_MEDIAN,64111.647093,2049.390423
176,NEW_CREDIT_TO_ANNUITY_RATIO,58020.846781,3437.932269
10,ANNUITY_div_CREDIT,43782.493319,2085.818931
21,CNT_INSTALMENT_FUTURE_mean,28700.184668,1002.936748
53,EXT_SOURCE_3,26012.237024,1238.414148
123,MONTHS_BALANCE_std,25684.363078,1503.995116
55,EXT_SOURCE_max,24841.509377,2060.070321
328,app_EXT_SOURCE_2_times_EXT_SOURCE_3,24837.987352,2127.586294


In [49]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [50]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [51]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.035506
1,100005,0.081070
2,100013,0.024152
3,100028,0.044840
4,100038,0.184463


In [52]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [53]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [54]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [55]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [56]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [57]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [58]:
# 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

# predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

# make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

### ### credit card blanceの読み込み

In [59]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [60]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [61]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [62]:
print(credit.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [63]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [64]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [65]:
df_train = pd.merge(app_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 430)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [66]:
# テーブル結合
df_test = pd.merge(app_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 429)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [67]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [68]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [69]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [70]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [71]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [72]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 949.01 MB
Memory usage of optimization is 527996515.00 MB
Decreased by -55636649.0%
Memory usage of dataframe is 150.06 MB
Memory usage of optimization is 83644832.00 MB
Decreased by -55742084.2%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [73]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [74]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 428) (61503, 428)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.765712	valid_1's auc: 0.748431
[200]	training's auc: 0.778519	valid_1's auc: 0.757154
[300]	training's auc: 0.789741	valid_1's auc: 0.763897
[400]	training's auc: 0.799004	valid_1's auc: 0.7687
[500]	training's auc: 0.806551	valid_1's auc: 0.771706
[600]	training's auc: 0.813066	valid_1's auc: 0.773591
[700]	training's auc: 0.818955	valid_1's auc: 0.774994
[800]	training's auc: 0.824419	valid_1's auc: 0.776233
[900]	training's auc: 0.829551	valid_1's auc: 0.777026
[1000]	training's auc: 0.83431	valid_1's auc: 0.777661
[1100]	training's auc: 0.838791	valid_1's auc: 0.778249
[1200]	training's auc: 0.84312	valid_1's auc: 0.778618
[1300]	training's auc: 0.84734	valid_1's auc: 0.778996
[1400]	training's auc: 0.851244	valid_1's auc: 0.779296
[1500]	training's auc: 0.855039	valid_1's auc

In [75]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
124,EXT_SOURCE_mean,357900.713639,3577.004933
262,NEW_EXT_SOURCES_GEO,142168.597623,1896.246273
256,NEW_CREDIT_TO_ANNUITY_RATIO,55470.175087,3545.614900
264,NEW_EXT_SOURCES_MEDIAN,54359.949152,1942.824597
58,ANNUITY_div_CREDIT,38839.464107,1930.261911
123,EXT_SOURCE_max,29543.733540,2587.650915
85,CNT_INSTALMENT_FUTURE_mean,25746.084448,1957.301732
195,MONTHS_BALANCE_std_y,24175.628328,1620.515287
121,EXT_SOURCE_3,23973.252711,1650.836870
265,NEW_EXT_SOURCES_PROD,23495.977674,2810.617692


In [76]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [77]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [78]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering3.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036257
1,100005,0.077221
2,100013,0.024883
3,100028,0.049665
4,100038,0.183508


### previous applicationの読み込み

In [79]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [80]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [81]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [82]:
print(previous.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [83]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [84]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [85]:
df_train = pd.merge(app_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 520)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [86]:
# テーブル結合
df_test = pd.merge(app_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 519)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [87]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [88]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [89]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [90]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [91]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [92]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1107.37 MB
Memory usage of optimization is 638700475.00 MB
Decreased by -57677139.1%
Memory usage of dataframe is 175.16 MB
Memory usage of optimization is 101192672.00 MB
Decreased by -57771795.2%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [93]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [94]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 518) (61503, 518)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.765774	valid_1's auc: 0.748311
[200]	training's auc: 0.778697	valid_1's auc: 0.757212
[300]	training's auc: 0.789643	valid_1's auc: 0.763738
[400]	training's auc: 0.798777	valid_1's auc: 0.768627
[500]	training's auc: 0.80643	valid_1's auc: 0.771533
[600]	training's auc: 0.813036	valid_1's auc: 0.773545
[700]	training's auc: 0.818988	valid_1's auc: 0.774978
[800]	training's auc: 0.824486	valid_1's auc: 0.7762
[900]	training's auc: 0.829607	valid_1's auc: 0.777104
[1000]	training's auc: 0.834447	valid_1's auc: 0.777764
[1100]	training's auc: 0.838927	valid_1's auc: 0.778107
[1200]	training's auc: 0.843257	valid_1's auc: 0.778402
[1300]	training's auc: 0.847377	valid_1's auc: 0.778596
[1400]	training's auc: 0.851326	valid_1's auc: 0.77881
[1500]	training's auc: 0.855032	valid_1's au

In [95]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
192,EXT_SOURCE_mean,352786.697783,5356.100643
342,NEW_EXT_SOURCES_GEO,140389.615211,3890.191698
336,NEW_CREDIT_TO_ANNUITY_RATIO,53486.627593,3754.068909
344,NEW_EXT_SOURCES_MEDIAN,45066.881992,2891.133891
106,ANNUITY_div_CREDIT,40155.707471,2908.017123
191,EXT_SOURCE_max,30404.484207,1733.521331
189,EXT_SOURCE_3,28324.734176,1306.409351
149,CNT_INSTALMENT_FUTURE_mean,26066.471029,1344.971626
265,MONTHS_BALANCE_std,24517.452384,1318.346901
170,DAYS_BIRTH,22974.215739,1916.819004


In [96]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [97]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [98]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering4.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.035149
1,100005,0.078028
2,100013,0.027956
3,100028,0.047872
4,100038,0.184289


### installments paymentsの読み込み

In [99]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [100]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [101]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [102]:
print(installments.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')


In [103]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [104]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [105]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [106]:
df_train = pd.merge(app_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 546)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [107]:
# テーブル結合
df_test = pd.merge(app_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 545)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [108]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [109]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [110]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [111]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [112]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [113]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1150.77 MB
Memory usage of optimization is 670681619.00 MB
Decreased by -58280849.3%
Memory usage of dataframe is 182.04 MB
Memory usage of optimization is 106262048.00 MB
Decreased by -58373161.0%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [114]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [115]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 544) (61503, 544)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.768296	valid_1's auc: 0.74952
[200]	training's auc: 0.781381	valid_1's auc: 0.758318
[300]	training's auc: 0.792932	valid_1's auc: 0.765247
[400]	training's auc: 0.802566	valid_1's auc: 0.770453
[500]	training's auc: 0.810524	valid_1's auc: 0.773852
[600]	training's auc: 0.817426	valid_1's auc: 0.776054
[700]	training's auc: 0.823775	valid_1's auc: 0.777755
[800]	training's auc: 0.829475	valid_1's auc: 0.779023
[900]	training's auc: 0.834744	valid_1's auc: 0.779951
[1000]	training's auc: 0.839685	valid_1's auc: 0.780567
[1100]	training's auc: 0.84438	valid_1's auc: 0.781064
[1200]	training's auc: 0.848724	valid_1's auc: 0.781513
[1300]	training's auc: 0.853042	valid_1's auc: 0.781752
[1400]	training's auc: 0.857087	valid_1's auc: 0.781931
[1500]	training's auc: 0.860966	valid_1's 

In [116]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
208,EXT_SOURCE_mean,350142.887250,6117.192710
358,NEW_EXT_SOURCES_GEO,151596.024510,4613.331522
352,NEW_CREDIT_TO_ANNUITY_RATIO,50669.651889,4291.799058
360,NEW_EXT_SOURCES_MEDIAN,46532.403845,3244.478959
114,ANNUITY_div_CREDIT,36228.473423,913.738732
82,AMT_PAYMENT_min,32595.000565,1953.843346
538,app_EXT_SOURCE_2_times_EXT_SOURCE_3,26311.410411,2948.229252
157,CNT_INSTALMENT_FUTURE_mean,25813.149772,1335.364236
207,EXT_SOURCE_max,23818.541006,2236.439459
205,EXT_SOURCE_3,23616.173122,856.680820


In [117]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [118]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [119]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering5.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.031053
1,100005,0.091903
2,100013,0.031174
3,100028,0.048822
4,100038,0.159629


### bureauの読み込み

In [128]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [129]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [130]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [131]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [132]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [133]:
bureau_ohe.head()

,SK_ID_CURR,SK_ID_BUREAU_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan_for_business_development,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending),CREDIT_TYPE_Loan_for_the_purchase_of_equipment,CREDIT_TYPE_Loan_for_working_capital_replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile_operator_loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real_estate_loan,CREDIT_TYPE_Unknown_type_of_loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [135]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [136]:
# infをnanへ置換
bureau_ohe_agg = bureau_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [137]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 626)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [138]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 625)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [139]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('EXT_SOURCE_3')
encoding_columns.remove('EXT_SOURCE_2')
encoding_columns.remove('EXT_SOURCE_1')

In [140]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [141]:
encoding_columns.append('EXT_SOURCE_3')
encoding_columns.append('EXT_SOURCE_2')
encoding_columns.append('EXT_SOURCE_1')

In [142]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [143]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [144]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.117323,-0.006439,-1.882730,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-0.187765,-1.255506
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.117323,-0.006439,-0.045919,-0.03285,-0.122220,-0.493725,-0.002629,0.0,0.304646,0.504252
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.117323,-0.006439,2.215201,-0.03285,-0.122220,-0.493725,-0.002629,0.0,-2.038484,-1.255506
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,-0.117323,-0.006439,0.405970,-0.03285,8.182009,-0.493725,-0.002629,0.0,-0.061310,0.504252
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.117323,-0.006439,-0.620244,-0.03285,-0.122220,1.480658,-0.002629,0.0,1.408814,1.246139


In [145]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()


Memory usage of dataframe is 1338.46 MB
Memory usage of optimization is 769085139.00 MB
Decreased by -57460214.9%
Memory usage of dataframe is 211.79 MB
Memory usage of optimization is 121860128.00 MB
Decreased by -57538118.7%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [146]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [147]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 624) (61503, 624)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.769512	valid_1's auc: 0.74975
[200]	training's auc: 0.784044	valid_1's auc: 0.759372
[300]	training's auc: 0.796833	valid_1's auc: 0.767178
[400]	training's auc: 0.80734	valid_1's auc: 0.772806
[500]	training's auc: 0.815749	valid_1's auc: 0.77642
[600]	training's auc: 0.823278	valid_1's auc: 0.779033
[700]	training's auc: 0.829832	valid_1's auc: 0.780976
[800]	training's auc: 0.83598	valid_1's auc: 0.782372
[900]	training's auc: 0.841751	valid_1's auc: 0.783428
[1000]	training's auc: 0.847076	valid_1's auc: 0.78437
[1100]	training's auc: 0.852093	valid_1's auc: 0.785104
[1200]	training's auc: 0.856921	valid_1's auc: 0.785675
[1300]	training's auc: 0.861559	valid_1's auc: 0.786177
[1400]	training's auc: 0.865922	valid_1's auc: 0.786554
[1500]	training's auc: 0.870198	valid_1's auc

In [148]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
285,EXT_SOURCE_mean,289495.109057,3768.681102
435,NEW_EXT_SOURCES_GEO,203348.008996,1985.831560
437,NEW_EXT_SOURCES_MEDIAN,56118.559326,2960.825632
429,NEW_CREDIT_TO_ANNUITY_RATIO,48690.040556,2440.012523
140,ANNUITY_div_CREDIT,33002.813404,2250.849289
108,AMT_PAYMENT_min,32414.899942,1150.586686
284,EXT_SOURCE_max,31510.085930,2291.194260
187,CNT_INSTALMENT_FUTURE_mean,26017.693279,968.411467
623,debt_credit_ratio_None,25980.987288,2108.633575
430,NEW_CREDIT_TO_GOODS_RATIO,21538.801309,705.811983


In [149]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [150]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [151]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering6.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.035476
1,100005,0.115863
2,100013,0.039537
3,100028,0.047273
4,100038,0.154564


### mean mean 

In [152]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [153]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [154]:
#
#app_train_mean1 = app_train_mean1.fillna(0)
#app_test_mean1 = app_test_mean1.fillna(0)

#app_train_mean2 = app_train_mean2.fillna(0)
#app_test_mean2 = app_test_mean2.fillna(0)

#app_train_mean3 = app_train_mean3.fillna(0)
#app_test_mean3 = app_test_mean3.fillna(0)

#app_train_mean4 = app_train_mean4.fillna(0)
#app_test_mean4 = app_test_mean4.fillna(0)

In [155]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_train = pd.merge(df_train, app_train_mean1, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean2, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean3, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean4, on='SK_ID_CURR', how='left')

df_train = df_train.fillna(0)

print(df_train.shape)
df_train.head()

(307511, 630)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,0.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.000000,0.000000,0.0,0.0,28.0,3.0,0.0000,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.000000,0.000000,0.0,0.0,4.0,1.0,0.0000,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.000000,0.015152,0.0,0.0,66.0,5.0,0.0000,0.0,0.0,0.0


In [156]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_test = pd.merge(df_test, app_test_mean1, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean2, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean3, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean4, on='SK_ID_CURR', how='left')

df_test = df_test.fillna(0)

print(df_test.shape)
df_test.head()

(48744, 629)


,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.000000,0.0,0.0,9.0,2.0,0.0,0.00000,0.000000,0.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.090909,0.0,0.0,11.0,1.0,0.0,0.00000,0.000000,0.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.027778,0.0,0.0,36.0,3.0,0.0,-1185.60375,0.000000,0.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.000000,0.0,0.0,31.0,2.0,0.0,0.00000,-1148.027459,0.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.000000,0.0,0.0,13.0,1.0,0.0,0.00000,0.000000,0.0


In [157]:
# 処理対象の列名をリスト化(Rev/Cashのcolumnを取得する)
replace_columns = list(df_test.select_dtypes(include='number').columns)
replace_columns.remove('SK_ID_CURR')
replace_columns.remove('EXT_SOURCE_3')
replace_columns.remove('EXT_SOURCE_2')
replace_columns.remove('EXT_SOURCE_1')

In [158]:
### 欠損値を補完する()

#for column in processing_columns:
#    #print("Missing Value Completion Process: " + column + "\n")
#    result = modules.process_imputer(df_train, df_test, replace_columns, column)
#    if result != True:
#        print("Can not Process: " + column + "\n")

In [159]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,0.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.000000,0.000000,0.0,0.0,28.0,3.0,0.0000,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.000000,0.000000,0.0,0.0,4.0,1.0,0.0000,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.000000,0.015152,0.0,0.0,66.0,5.0,0.0000,0.0,0.0,0.0


In [160]:
### 欠損値を補完する
#df_train = df_train.fillna(df_train.max() + (df_train.max() * 0.4))
#df_train.head()

In [161]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,-5170.0,...,0.0,0.000000,0.0,0.0,9.0,2.0,0.0,0.00000,0.000000,0.0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,-9118.0,...,0.0,0.090909,0.0,0.0,11.0,1.0,0.0,0.00000,0.000000,0.0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,-2175.0,...,0.0,0.027778,0.0,0.0,36.0,3.0,0.0,-1185.60375,0.000000,0.0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,-2000.0,...,0.0,0.000000,0.0,0.0,31.0,2.0,0.0,0.00000,-1148.027459,0.0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,-4000.0,...,0.0,0.000000,0.0,0.0,13.0,1.0,0.0,0.00000,0.000000,0.0


In [162]:
### 欠損値補完
#df_test = df_test.fillna(df_test.max() + (df_test.max() * 0.4))
#df_test.head()

In [163]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
#encoding_columns.remove('EXT_SOURCE_3')
#encoding_columns.remove('EXT_SOURCE_2')
#encoding_columns.remove('EXT_SOURCE_1')

In [164]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [165]:
#encoding_columns.append('EXT_SOURCE_3')
#encoding_columns.append('EXT_SOURCE_2')
#encoding_columns.append('EXT_SOURCE_1')

In [166]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [167]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,0.171087,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,1.122551,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,-0.007087,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,-1.245078,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,-0.021335,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [168]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.653139,0.647486,-0.284044,-0.203186,-0.507253,0.047087,1.597495,0.808839,...,-0.118523,-0.476503,-0.00255,0.0,-0.118159,-0.856749,-3.049664,0.086846,0.222678,0.011144
1,100003,0,-0.653139,1.212688,1.495411,0.526520,1.580121,-1.730597,-0.222770,0.439156,...,-0.118523,-0.476503,-0.00255,0.0,0.298617,0.462282,0.328005,0.086846,0.222678,0.011144
2,100004,0,-0.653139,-1.655798,-1.648326,-1.231482,-1.005031,-0.863129,-0.716560,1.556409,...,-0.118523,-0.476503,-0.00255,0.0,-1.311051,-0.856749,0.328005,0.086846,0.222678,0.011144
3,100006,0,-0.653139,-0.175278,-0.635708,0.120608,-0.637240,-1.112298,-0.707915,-0.026918,...,8.437188,-0.476503,-0.00255,0.0,-0.015781,0.462282,-3.049843,0.086846,0.222678,0.011144
4,100007,0,-0.653139,-0.394208,0.042354,-0.381375,-0.101780,0.800124,-0.901472,-0.026771,...,-0.118523,1.595367,-0.00255,0.0,1.433840,1.232306,0.328005,0.086846,0.222678,0.011144


In [169]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 1347.85 MB
Memory usage of optimization is 774005315.00 MB
Decreased by -57425182.6%
Memory usage of dataframe is 213.28 MB
Memory usage of optimization is 122640032.00 MB
Decreased by -57502481.9%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,-0.65332,0.647461,-0.283936,-0.203247,-0.507324,0.047089,1.597656,0.809082,...,-0.11853,-0.476562,-0.00255,0.0,-0.118164,-0.856934,-3.048828,0.086853,0.222656,0.011147
1,100003,0,-0.65332,1.212891,1.495117,0.526367,1.580078,-1.730469,-0.222778,0.439209,...,-0.11853,-0.476562,-0.00255,0.0,0.298584,0.462402,0.328125,0.086853,0.222656,0.011147
2,100004,0,-0.65332,-1.656250,-1.648438,-1.231445,-1.004883,-0.863281,-0.716797,1.556641,...,-0.11853,-0.476562,-0.00255,0.0,-1.311523,-0.856934,0.328125,0.086853,0.222656,0.011147
3,100006,0,-0.65332,-0.175293,-0.635742,0.120605,-0.637207,-1.112305,-0.708008,-0.026917,...,8.43750,-0.476562,-0.00255,0.0,-0.015778,0.462402,-3.050781,0.086853,0.222656,0.011147
4,100007,0,-0.65332,-0.394287,0.042358,-0.381348,-0.101807,0.800293,-0.901367,-0.026764,...,-0.11853,1.595703,-0.00255,0.0,1.433594,1.232422,0.328125,0.086853,0.222656,0.011147


In [170]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [171]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 628) (61503, 628)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.768197	valid_1's auc: 0.749952
[200]	training's auc: 0.783017	valid_1's auc: 0.760214
[300]	training's auc: 0.796056	valid_1's auc: 0.768142
[400]	training's auc: 0.806453	valid_1's auc: 0.773562
[500]	training's auc: 0.814891	valid_1's auc: 0.777282
[600]	training's auc: 0.822397	valid_1's auc: 0.779848
[700]	training's auc: 0.829014	valid_1's auc: 0.78188
[800]	training's auc: 0.835008	valid_1's auc: 0.78335
[900]	training's auc: 0.840673	valid_1's auc: 0.784469
[1000]	training's auc: 0.846003	valid_1's auc: 0.785438
[1100]	training's auc: 0.850952	valid_1's auc: 0.786097
[1200]	training's auc: 0.855688	valid_1's auc: 0.786559
[1300]	training's auc: 0.860127	valid_1's auc: 0.786859
[1400]	training's auc: 0.86446	valid_1's auc: 0.787281
[1500]	training's auc: 0.868516	valid_1's a

[2300]	training's auc: 0.897936	valid_1's auc: 0.78609
[2400]	training's auc: 0.900997	valid_1's auc: 0.786111
[2500]	training's auc: 0.903847	valid_1's auc: 0.7863
[2600]	training's auc: 0.906644	valid_1's auc: 0.786343
[2700]	training's auc: 0.909486	valid_1's auc: 0.786418
[2800]	training's auc: 0.912135	valid_1's auc: 0.786428
[auc] tr:0.911148, va0.7865
-------------------- result --------------------
[[0.         0.9141312  0.78923305]
 [1.         0.91616303 0.79478045]
 [2.         0.90451507 0.79263255]
 [3.         0.88780837 0.79130726]
 [4.         0.91114776 0.78646016]]
[cv] tr:0.9068+-0.0103, va:0.7909+-0.0029
[oof]0.7909


In [172]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
285,EXT_SOURCE_mean,363719.918992,5892.906498
435,NEW_EXT_SOURCES_GEO,129496.859260,5189.145577
437,NEW_EXT_SOURCES_MEDIAN,58386.748389,2862.934092
429,NEW_CREDIT_TO_ANNUITY_RATIO,48087.582596,3964.910998
140,ANNUITY_div_CREDIT,37660.669162,2402.113036
284,EXT_SOURCE_max,28801.524707,1862.297933
187,CNT_INSTALMENT_FUTURE_mean,25651.611565,781.000846
0,AMT_ANNUITY,22759.453500,1709.591045
108,AMT_PAYMENT_min,22710.329138,2220.053262
238,DAYS_BIRTH,22201.689936,1452.527291


In [173]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [174]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [175]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering7.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036222
1,100005,0.135552
2,100013,0.040300
3,100028,0.050022
4,100038,0.147079


### 変数選択001

In [176]:
length = int(len(df_train.columns) * 0.95)
length

598

In [177]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

aa
bb


In [178]:
df_train = df_train[train_columns]
df_test = df_test[test_columns]
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 702.96 MB
Memory usage of optimization is 737103995.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 111.38 MB
Memory usage of optimization is 116790752.00 MB
Decreased by -104857500.0%


,EXT_SOURCE_mean,NEW_EXT_SOURCES_GEO,NEW_EXT_SOURCES_MEDIAN,NEW_CREDIT_TO_ANNUITY_RATIO,ANNUITY_div_CREDIT,EXT_SOURCE_max,CNT_INSTALMENT_FUTURE_mean,AMT_ANNUITY,AMT_PAYMENT_min,DAYS_BIRTH,...,ORGANIZATION_TYPE_Insurance,NAME_CONTRACT_TYPE_nan,ORGANIZATION_TYPE_Legal_Services,CREDIT_TYPE_Another_type_of_loan_mean,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Religion,CREDIT_DAY_OVERDUE_min,CREDIT_CURRENCY_nan_mean,SK_ID_CURR,TARGET
0,-1.963867,-1.798828,-1.984375,-0.696289,0.276123,-1.943359,1.053711,-0.203247,1.005859,1.597656,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100002,1
1,-0.384766,-0.392822,-0.397705,2.062500,-1.100586,-0.103394,-0.263184,0.526367,0.860840,-0.222778,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100003,0
2,0.902832,0.920410,0.830078,-0.307373,-0.183105,0.746582,-1.155273,-1.231445,0.767578,-0.716797,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100004,0
3,0.966309,1.023438,0.890625,-1.231445,1.821289,0.104248,0.237305,0.120605,0.455078,-0.708008,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100006,0
4,-1.225586,-1.027344,-1.198242,0.119629,-0.491211,-1.726562,0.286377,-0.381348,-1.541992,-0.901367,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,60.125000,-0.021866,0.0,100007,0


In [179]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 702.96 MB
Memory usage of optimization is 737103995.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 111.38 MB
Memory usage of optimization is 116790752.00 MB
Decreased by -104857500.0%


,EXT_SOURCE_mean,NEW_EXT_SOURCES_GEO,NEW_EXT_SOURCES_MEDIAN,NEW_CREDIT_TO_ANNUITY_RATIO,ANNUITY_div_CREDIT,EXT_SOURCE_max,CNT_INSTALMENT_FUTURE_mean,AMT_ANNUITY,AMT_PAYMENT_min,DAYS_BIRTH,...,ORGANIZATION_TYPE_Insurance,NAME_CONTRACT_TYPE_nan,ORGANIZATION_TYPE_Legal_Services,CREDIT_TYPE_Another_type_of_loan_mean,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Religion,CREDIT_DAY_OVERDUE_min,CREDIT_CURRENCY_nan_mean,SK_ID_CURR,TARGET
0,-1.963867,-1.798828,-1.984375,-0.696289,0.276123,-1.943359,1.053711,-0.203247,1.005859,1.597656,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100002,1
1,-0.384766,-0.392822,-0.397705,2.062500,-1.100586,-0.103394,-0.263184,0.526367,0.860840,-0.222778,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100003,0
2,0.902832,0.920410,0.830078,-0.307373,-0.183105,0.746582,-1.155273,-1.231445,0.767578,-0.716797,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100004,0
3,0.966309,1.023438,0.890625,-1.231445,1.821289,0.104248,0.237305,0.120605,0.455078,-0.708008,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,-0.016632,-0.021866,0.0,100006,0
4,-1.225586,-1.027344,-1.198242,0.119629,-0.491211,-1.726562,0.286377,-0.381348,-1.541992,-0.901367,...,-0.044098,0.0,-0.031494,-0.051483,-0.032135,60.125000,-0.021866,0.0,100007,0


In [180]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [181]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 598) (61503, 598)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.768274	valid_1's auc: 0.749998
[200]	training's auc: 0.783121	valid_1's auc: 0.760233
[300]	training's auc: 0.796175	valid_1's auc: 0.768208
[400]	training's auc: 0.806616	valid_1's auc: 0.773701
[500]	training's auc: 0.815007	valid_1's auc: 0.777299
[600]	training's auc: 0.822338	valid_1's auc: 0.779895
[700]	training's auc: 0.828832	valid_1's auc: 0.781862
[800]	training's auc: 0.83486	valid_1's auc: 0.783311
[900]	training's auc: 0.840581	valid_1's auc: 0.784464
[1000]	training's auc: 0.84595	valid_1's auc: 0.785499
[1100]	training's auc: 0.850817	valid_1's auc: 0.786128
[1200]	training's auc: 0.855509	valid_1's auc: 0.786684
[1300]	training's auc: 0.859998	valid_1's auc: 0.786979
[1400]	training's auc: 0.864264	valid_1's auc: 0.787399
[1500]	training's auc: 0.868304	valid_1's 

[2100]	training's auc: 0.891277	valid_1's auc: 0.786066
[2200]	training's auc: 0.894543	valid_1's auc: 0.78607
[2300]	training's auc: 0.897777	valid_1's auc: 0.786214
[2400]	training's auc: 0.900765	valid_1's auc: 0.786248
[auc] tr:0.899278, va0.7863
-------------------- result --------------------
[[0.         0.92098209 0.78926209]
 [1.         0.90153978 0.79419354]
 [2.         0.90836401 0.79317326]
 [3.         0.89780134 0.79168451]
 [4.         0.89927777 0.7862955 ]]
[cv] tr:0.9056+-0.0085, va:0.7909+-0.0028
[oof]0.7909


In [182]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
277,EXT_SOURCE_mean,298059.260837,5755.134637
426,NEW_EXT_SOURCES_GEO,172221.202667,5516.139021
428,NEW_EXT_SOURCES_MEDIAN,68636.302515,1483.766735
420,NEW_CREDIT_TO_ANNUITY_RATIO,50348.811556,3038.679629
276,EXT_SOURCE_max,40707.577385,2812.194605
140,ANNUITY_div_CREDIT,35296.410475,1469.610771
187,CNT_INSTALMENT_FUTURE_mean,25700.266422,1226.824082
0,AMT_ANNUITY,23019.393967,2054.084096
278,EXT_SOURCE_min,22868.449735,2614.759089
108,AMT_PAYMENT_min,22246.647939,1227.834249


In [183]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [184]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [185]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering8.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036470
1,100005,0.133856
2,100013,0.038518
3,100028,0.054370
4,100038,0.146074


## パラメータ・チューニング

In [186]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-29 00:30:11,383] A new study created in memory with name: no-name-790d0528-4d76-4b92-b857-0671f7ab03dd
[I 2022-06-29 00:49:43,727] Trial 0 finished with value: 0.789262090995524 and parameters: {'num_leaves': 153, 'min_child_samples': 60, 'min_sum_hessian_in_leaf': 0.0006650872452558587, 'feature_fraction': 0.8382013426865338, 'bagging_fraction': 0.7111857053220034, 'lambda_l1': 0.04783548481564155, 'lambda_l2': 83.41839564147887}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 00:49:57,972] Trial 4 finished with value: 0.789262090995524 and parameters: {'num_leaves': 179, 'min_child_samples': 132, 'min_sum_hessian_in_leaf': 0.00010447550132245846, 'feature_fraction': 0.8173798892605688, 'bagging_fraction': 0.9795640877240517, 'lambda_l1': 0.15639356432720944, 'lambda_l2': 83.07085203603161}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 00:50:32,710] Trial 2 finished with value: 0.789262090995524 and parameters: {'num_leaves': 249, 'min_child_sa

[I 2022-06-29 01:58:53,090] Trial 21 finished with value: 0.789262090995524 and parameters: {'num_leaves': 30, 'min_child_samples': 128, 'min_sum_hessian_in_leaf': 0.008823477575346428, 'feature_fraction': 0.8869988045256659, 'bagging_fraction': 0.7841671732246611, 'lambda_l1': 0.14928162908169837, 'lambda_l2': 18.81201829732441}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 01:59:03,684] Trial 22 finished with value: 0.789262090995524 and parameters: {'num_leaves': 87, 'min_child_samples': 124, 'min_sum_hessian_in_leaf': 0.002929352443636569, 'feature_fraction': 0.918019697750742, 'bagging_fraction': 0.7840650780739538, 'lambda_l1': 0.07127459516347671, 'lambda_l2': 0.3993505399140104}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 01:59:25,398] Trial 23 finished with value: 0.789262090995524 and parameters: {'num_leaves': 90, 'min_child_samples': 128, 'min_sum_hessian_in_leaf': 4.3534647768397515e-05, 'feature_fraction': 0.886900486036355, 'bagging_frac

[I 2022-06-29 03:07:20,445] Trial 41 finished with value: 0.789262090995524 and parameters: {'num_leaves': 8, 'min_child_samples': 7, 'min_sum_hessian_in_leaf': 0.005856566760444814, 'feature_fraction': 0.93346973275941, 'bagging_fraction': 0.6982484823268215, 'lambda_l1': 0.020477692333448594, 'lambda_l2': 12.149413297699521}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 03:07:55,664] Trial 43 finished with value: 0.789262090995524 and parameters: {'num_leaves': 98, 'min_child_samples': 113, 'min_sum_hessian_in_leaf': 0.00657838287295069, 'feature_fraction': 0.8732187309779104, 'bagging_fraction': 0.8398411135040066, 'lambda_l1': 0.12198944407545113, 'lambda_l2': 19.908016119116873}. Best is trial 0 with value: 0.789262090995524.
[I 2022-06-29 03:09:02,351] Trial 44 finished with value: 0.789262090995524 and parameters: {'num_leaves': 96, 'min_child_samples': 114, 'min_sum_hessian_in_leaf': 0.001198442575171534, 'feature_fraction': 0.8721177172657749, 'bagging_fraction

In [187]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

auc(best)=0.7893


{'num_leaves': 153,
 'min_child_samples': 60,
 'min_sum_hessian_in_leaf': 0.0006650872452558587,
 'feature_fraction': 0.8382013426865338,
 'bagging_fraction': 0.7111857053220034,
 'lambda_l1': 0.04783548481564155,
 'lambda_l2': 83.41839564147887}

In [188]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 153,
 'min_child_samples': 165,
 'min_sum_hessian_in_leaf': 0.0006650872452558587,
 'feature_fraction': 0.8382013426865338,
 'bagging_fraction': 0.7111857053220034,
 'lambda_l1': 0.04783548481564155,
 'lambda_l2': 83.41839564147887,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'nthread': -1,
 'n_estimators': 5000,
 'learning_rate': 0.01,
 'max_depth': 11,
 'colsample_bytree': 0.613,
 'subsample': 0.708,
 'max_bin': 407,
 'reg_alpha': 3.564,
 'reg_lambda': 4.93,
 'silent': -1,
 'verbose': -1,
 'importance_type': 'gain'}

In [189]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

-------------------- 0 --------------------
(246008, 598) (61503, 598)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] feature_fraction is set=0.8382013426865338, colsample_bytree=0.613 will be ignored. Current value: feature_fraction=0.8382013426865338
[LightGBM] [Warning] lambda_l1 is set=0.04783548481564155, reg_alpha=3.564 will be ignored. Current value: lambda_l1=0.04783548481564155
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.0006650872452558587, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.0006650872452558587
[LightGBM] [Warning] bagging_fraction is set=0.7111857053220034, subsample=0.708 will be ignored. Current value: bagging_fraction=0.7111857053220034
[LightGBM] [Warning] lambda_l2 is set=83.41839564147887, reg_lambda=4.93 will be ignored. Current value: lambda_l2=83.41839564147887
[100]	training's auc: 0.780915	valid_1's auc: 0.751715
[200]	tra

[100]	training's auc: 0.781088	valid_1's auc: 0.755109
[200]	training's auc: 0.798101	valid_1's auc: 0.763682
[300]	training's auc: 0.81267	valid_1's auc: 0.770433
[400]	training's auc: 0.824874	valid_1's auc: 0.775363
[500]	training's auc: 0.834887	valid_1's auc: 0.778692
[600]	training's auc: 0.843746	valid_1's auc: 0.780873
[700]	training's auc: 0.851593	valid_1's auc: 0.782614
[800]	training's auc: 0.858557	valid_1's auc: 0.784157
[900]	training's auc: 0.864827	valid_1's auc: 0.785337
[1000]	training's auc: 0.870964	valid_1's auc: 0.786349
[1100]	training's auc: 0.876577	valid_1's auc: 0.787118
[1200]	training's auc: 0.881376	valid_1's auc: 0.787618
[1300]	training's auc: 0.885809	valid_1's auc: 0.788142
[1400]	training's auc: 0.890311	valid_1's auc: 0.788615
[1500]	training's auc: 0.894285	valid_1's auc: 0.788963
[1600]	training's auc: 0.898115	valid_1's auc: 0.789249
[1700]	training's auc: 0.90171	valid_1's auc: 0.789498
[1800]	training's auc: 0.904942	valid_1's auc: 0.789646
[19

In [190]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.
(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.034005
1,100005,0.124841
2,100013,0.046288
3,100028,0.050267
4,100038,0.146359


In [191]:
del df_train
del df_test
gc.collect()

250